## Models Evaluations

In [1]:
import numpy as np
import tensorflow as tf

2025-02-06 22:27:57.549136: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
import utils

In [2]:
def load_model(model_path):
    model = tf.keras.models.load_model(model_path)
    return model

### Preparing data

In [7]:
UNKNOWN_TOKEN = "<UNK>"

In [6]:
word_to_index, index_to_word, vocabs = utils.word_indexing("word_to_index.npy")

In [8]:
word_to_index[UNKNOWN_TOKEN]

131

In [4]:
# Read limited tokens
with open("cleaned_tokens.txt", "r") as f:
    cleaned_tokens = f.read().split()

In [10]:
tokenized = []
for w in cleaned_tokens:
    if w not in word_to_index:
        tokenized.append(word_to_index[UNKNOWN_TOKEN])
    else:
        tokenized.append(word_to_index[w])

In [11]:
def generate_ngram_data(tokenized, n):
    """Generate N-gram data for the language model."""
    inputs, targets = [], []
    for i in range(len(tokenized) - n):
        inputs.append(tokenized[i:i + n])
        targets.append(tokenized[i + n])
    return np.array(inputs), np.array(targets)

In [12]:
inputs, targets = generate_ngram_data(tokenized, utils.N)

In [14]:
from tensorflow.keras.utils import to_categorical

# Convert targets to one-hot encoding
targets_one_hot = to_categorical(targets, num_classes=len(vocabs))

In [15]:
# Split data into training and validation sets
from sklearn.model_selection import train_test_split
trained_inputs, val_inputs, trained_targets, val_targets, trained_targets_one_hot, val_targets_one_hot = train_test_split(inputs, targets, targets_one_hot, test_size=0.2)

### Loading the models

In [21]:
skip_model1 = load_model("model_word_prediction1.keras")
skip_model2 = load_model("model_word_prediction2.keras")
scratch_model1 = load_model("model_scratch_word_prediction1.keras")
scratch_model2 = load_model("model_scratch_word_prediction2.keras")
extended_model1 = load_model("pre_model_word_prediction1.keras")

In [22]:
skip_model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 50)             8750      
                                                                 
 flatten (Flatten)           (None, 250)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               128512    
                                                                 
 dense_2 (Dense)             (None, 175)               89775     
                                                                 
Total params: 227037 (886.86 KB)
Trainable params: 218287 (852.68 KB)
Non-trainable params: 8750 (34.18 KB)
_________________________________________________________________


In [25]:
loss, _ = skip_model1.evaluate(val_inputs, val_targets)
print("Skip model 1 loss: ", loss)
pp = np.exp(loss)
print("Perplexity: ", pp)

57/57 [==============================] - 0s 3ms/step - loss: 1.1386 - accuracy: 0.7138
Skip model 1 loss:  1.1386157274246216
Perplexity:  3.1224430598801085


In [26]:
loss, _ = skip_model2.evaluate(val_inputs, val_targets)
print("Skip model 2 loss: ", loss)
pp = np.exp(loss)
print("Perplexity: ", pp)

57/57 [==============================] - 1s 8ms/step - loss: 0.2038 - accuracy: 0.9285
Skip model 2 loss:  0.20379571616649628
Perplexity:  1.2260476661545283
